In [ ]:
import os
import shutil
import subprocess
import numpy as np
import xarray as xr
import netCDF4 as nc
from tqdm import tqdm, trange
from joblib import Parallel, delayed
import calendar
from pathlib import Path

# configuration

ROOT_DIR = Path("../data").resolve()
RUN_DIR = ROOT_DIR / "run"
SHP_DIR = ROOT_DIR / "Shp"
D_DIR = RUN_DIR / "reset_annual"
RES = "500m"

RES_PATH = ROOT_DIR / f"{RES}.txt"
region = [-180,180,-60,90]

os.makedirs(D_DIR, exist_ok=True)
os.chdir(RUN_DIR)

cdo_command = "cdo -O -L -b F32 -f nc4 -z zip_3"
# Set cdo function to use parallel operations 
def cdo_mul(filename1, filename2, filename3):
    subprocess.run(f"{cdo_command} -mul {filename1} {filename2} {filename3}", shell=True, check=True)

def cdo_sub(filename1, filename2, filename3):
    subprocess.run(f"{cdo_command} -sub {filename1} {filename2} {filename3}", shell=True, check=True)
    
def cdo_expr(filename1, filename2, nday):
    subprocess.run(f'{cdo_command} -expr,"Ee=Dr*1000*2257/(3600*24*{nday})" {filename1} {filename2}', shell=True, check=True)

def cdo_setvals(filename1, filename2):
    subprocess.run(f'{cdo_command} -setvals,0,nan {filename1} {filename2}', shell=True, check=True)

def cdo_setrtoc(filename1, filename2):
    subprocess.run(f'{cdo_command} -setrtoc,1e-1,inf,1 -setrtoc,-inf,1e-1,nan {filename1} {filename2}', shell=True, check=True)

def dp_Dr():
    # remap the 0p1 resolution to 0p1 resolution(no need, but for the sake of formatting consistency)
    for j in range(18):
        print(f"year is {j+2003}")
        subprocess.run(f"{cdo_command} -P 48 --no_remap_weights -remapbil,{RES_PATH} {D_DIR}/Dr_{2003+j}_tmp1.nc {D_DIR}/Dr_{2003+j}_tmp2.nc", shell=True, check=True)
    Parallel(n_jobs=5)(delayed(cdo_mul)(f"{D_DIR}/Dr_{2003+j}_tmp2.nc", "mask_combine_all.nc", f"{D_DIR}/Dr_{2003+j}.nc") for j in tqdm(range(18)))

def dp_Dbedrock():
    # remap the 0p1 resolution to 0p1 resolution(no need, but for the sake of formatting consistency)
    for j in range(18):
        print(f"year is {j+2003}")
        subprocess.run(f"{cdo_command} -P 48 --no_remap_weights -remapbil,{RES_PATH} {D_DIR}/Dbedrock_{2003+j}_tmp1.nc {D_DIR}/Dbedrock_{2003+j}_tmp2.nc", shell=True, check=True)

    Parallel(n_jobs=5)(delayed(cdo_mul)(f"{D_DIR}/Dbedrock_{2003+j}_tmp2.nc", "mask_combine_all.nc", f"{D_DIR}/Dbedrock_{2003+j}.nc") for j in tqdm(range(18)))


def dp_Dbedrock_Frequency():
    # calculate Dbedrock Frequency
    s1 = 0
    for year in range(2003,2021):
        file = f'{D_DIR}/Dbedrock_{year}_tmp2.nc'
        image = xr.open_dataset(file)
        s = image['Dbedrock']

        print(f'year: {year}, min: {s.min().values}, max: {s.max().values}')
        
        s = np.where(s > 0, 2, np.where(s < 0, 1, 0))
        print(f'year: {year}, min: {s.min()}, max: {s.max()}')
        
        print(s.min(),s.max())
        
        if year == 2003:
            s1 = s
        else:
            s1 = s*s1
        
        s_nonan = np.where((s1<0), 0, s1)
        print(s1.min(),s1.max(),np.mean(s_nonan))
        image.close()
    print('end do')

    s1 = np.where((s1 >=2) & (s1 < 262144), 2, s1)
    s1 = np.where((s1 == 1), 3, s1)
    s1 = np.where(s1==262144, 1, s1)

    file_mask = f'mask12.nc'
    mask = xr.open_dataset(file_mask)
    s2 = mask['Band1']
    print(s1.min(),s1.max())

    s1 = np.where((s1==0) & (s2 == 1), 4, s1)

    print(s1.min(),s1.max())
    shutil.copyfile('{D_DIR}/Dbedrock_2003.nc', 'Dbedrock_Frequency_tmp1.nc')

    with nc.Dataset('Dbedrock_Frequency_tmp1.nc', 'a') as file:
        s_var = file.variables['Dbedrock']
        new_s_data = s1 
        s_var[:,:] = new_s_data

    os.system('cdo mul Dbedrock_Frequency_tmp1.nc mask_combine_all.nc Dbedrock_Frequency.nc')


def dp_frequency_per_year():
    os.system(f'{cdo_command} -setrtoc,-0.5,0.5,nan D_frequency_per_year_tmp1.nc D_frequency_per_year_set0_to_nan_tmp1.nc')
    os.system(f'{cdo_command} timmean D_frequency_per_year_set0_to_nan_tmp1.nc D_frequency_per_year_mean_tmp1.nc')
    os.system(f'{cdo_command} timmax D_frequency_per_year_set0_to_nan_tmp1.nc D_frequency_per_year_max_tmp1.nc')

    os.system(f'{cdo_command} -P 48 --no_remap_weights -remapbil,{RES_PATH} D_frequency_per_year_mean_tmp1.nc D_frequency_per_year_mean_tmp2.nc')
    os.system(f"{cdo_command} mul D_frequency_per_year_mean_tmp2.nc mask_combine_all.nc D_frequency_per_year_mean.nc")

    os.system(f'{cdo_command} -P 48 --no_remap_weights -remapbil,{RES_PATH} D_frequency_per_year_max_tmp1.nc D_frequency_per_year_max_tmp2.nc')
    os.system(f"{cdo_command} mul D_frequency_per_year_max_tmp2.nc mask_combine_all.nc D_frequency_per_year_max.nc")

def dp_sum_frequency():
    os.system(f'{cdo_command} -setrtoc,-0.5,0.5,nan D_sum_frequency_tmp1.nc D_sum_frequency_tmp2.nc')
    os.system(f'{cdo_command} -P 48 --no_remap_weights -remapbil,{RES_PATH} D_sum_frequency_tmp2.nc D_sum_frequency_tmp3.nc')
    os.system(f"{cdo_command} mul D_sum_frequency_tmp3.nc mask_combine_all.nc D_sum_frequency.nc")

def dp_duration_per_year():
    os.system(f'{cdo_command} -setrtoc,-0.5,0.5,nan D_duration_per_year_tmp1.nc D_duration_per_year_set0_to_nan_tmp1.nc')
    os.system(f'{cdo_command} timmean D_duration_per_year_set0_to_nan_tmp1.nc D_duration_per_year_mean_tmp1.nc')
    os.system(f'{cdo_command} timmax D_duration_per_year_set0_to_nan_tmp1.nc D_duration_per_year_max_tmp1.nc')

    os.system(f'{cdo_command} -P 48 --no_remap_weights -remapbil,{RES_PATH} D_duration_per_year_mean_tmp1.nc D_duration_per_year_mean_tmp2.nc')
    os.system(f"{cdo_command} mul D_duration_per_year_mean_tmp2.nc mask_combine_all.nc D_duration_per_year_mean.nc")

    os.system(f'{cdo_command} -P 48 --no_remap_weights -remapbil,{RES_PATH} D_duration_per_year_max_tmp1.nc D_duration_per_year_max_tmp2.nc')
    os.system(f"{cdo_command} mul D_duration_per_year_max_tmp2.nc mask_combine_all.nc D_duration_per_year_max.nc")


def dp_duration_per_use():
    os.system(f'{cdo_command} -setrtoc,-0.5,0.5,nan D_duration_per_use_max_tmp1.nc D_duration_per_use_max_tmp2.nc')
    os.system(f'{cdo_command} -P 48 --no_remap_weights -remapbil,{RES_PATH} D_duration_per_use_max_tmp2.nc D_duration_per_use_max_tmp3.nc')
    os.system(f"{cdo_command} mul D_duration_per_use_max_tmp3.nc mask_combine_all.nc D_duration_per_use_max.nc")

    os.system(f'{cdo_command} -setrtoc,-0.5,0.5,nan D_duration_per_use_mean_tmp1.nc D_duration_per_use_mean_tmp2.nc')
    os.system(f'{cdo_command} -P 48 --no_remap_weights -remapbil,{RES_PATH} D_duration_per_use_mean_tmp2.nc D_duration_per_use_mean_tmp3.nc')
    os.system(f"{cdo_command} mul D_duration_per_use_mean_tmp3.nc mask_combine_all.nc D_duration_per_use_mean.nc")

def dp_sum_duration():
    os.system(f'{cdo_command} -setrtoc,-0.5,0.5,nan D_sum_duration_tmp1.nc D_sum_duration_tmp2.nc')
    os.system(f'{cdo_command} -P 48 --no_remap_weights -remapbil,{RES_PATH} D_sum_duration_tmp2.nc D_sum_duration_tmp3.nc')
    os.system(f"{cdo_command} mul D_sum_duration_tmp3.nc mask_combine_all.nc D_sum_duration.nc")

def dp_first_day():
    os.system(f'{cdo_command} -setrtoc,-0.5,0.5,nan D_first_day_tmp1.nc D_first_day_set0_to_nan_tmp1.nc')
    os.system(f'{cdo_command} timmax D_first_day_set0_to_nan_tmp1.nc D_first_day_max_tmp1.nc')
    os.system(f'{cdo_command} timmin D_first_day_set0_to_nan_tmp1.nc D_first_day_min_tmp1.nc')
    os.system(f'{cdo_command} sub D_first_day_max_tmp1.nc D_first_day_min_tmp1.nc D_first_day_max_sub_min_tmp1.nc')

    os.system(f'{cdo_command} -P 48 --no_remap_weights -remapbil,{RES_PATH} D_first_day_min_tmp1.nc D_first_day_min_tmp2.nc')
    os.system(f"{cdo_command} mul D_first_day_min_tmp2.nc mask_combine_all.nc D_first_day_min.nc")


    os.system(f'{cdo_command} -P 48 --no_remap_weights -remapbil,{RES_PATH} D_first_day_max_sub_min_tmp1.nc D_first_day_max_sub_min_tmp2.nc')
    os.system(f"{cdo_command} mul D_first_day_max_sub_min_tmp2.nc mask_combine_all.nc D_first_day_max_sub_min.nc")

if __name__=='__main__':
    dp_Dr()
    dp_Dbedrock()
    dp_Dbedrock_Frequency()

    dp_frequency_per_year()
    dp_sum_frequency()
    dp_duration_per_year()
    dp_duration_per_use()
    dp_sum_duration()
    dp_first_day()